# Track log evaluation

## Data
* [lag_data](https://drive.google.com/open?id=1Hc5bGXMLV3LqfnAwWKMPylfHothAHtPc)
* [lag_data1](https://drive.google.com/open?id=1ok6RiGLbfNGN-GYA3ejpwKE1TI7LSotE)

## Classification vs accumulation

![results](./image/Classification_results_s1_real.png)

```
wo classification accuracy: 50.0
w classification accuracy: 75.9493670886
wo classification accuracy: 39.0 / 78.0
w classification accuracy: 60.0 / 79.0
```

## Classification vs accumulated frame

![results](./image/results2.png)

```
results 1: 303.0 / 611.0 
results 2: 163.0 / 270.0 
results 3: 101.0 / 188.0 
results 4: 92.0 / 141.0 
results 5: 88.0 / 109.0 
results 6: 73.0 / 84.0 
results 7: 59.0 / 65.0 
results 8: 36.0 / 41.0 
```

In [1]:
import os, sys

import re

import matplotlib.pyplot as plt

import numpy as np


In [2]:
def track_log_parsing(file_path):
    
    track = {}
    track['id'] = []
    track['status'] = []
    track['skippedframe'] = []
    track['updatedframe'] = []
    track['accumulatedframe'] = []
    track['scopeframe'] = []
    track['x'] = []
    track['y'] = []
    track['z'] = []
    track['class'] = []
    track['pointnum'] = [] 
    track['timestamp'] = []
    
    f = open(file_path, 'r')
    lines = f.readlines()[1:]
    
    for line in lines:
        data = line.split(', ')
        
        for d in data:
            track[d.split(': ')[0]].append( float(d.split(': ')[1]) )
        
    f.close()
    
    return track

In [18]:
""" Classification vs time """

""" Path """

track_log_path_list = []

tlp = os.path.abspath('./log_data9/wo/log_track_0.txt')
track_log_path_list.append(tlp)

tlp = os.path.abspath('./log_data9/w/log_track_0.txt')
track_log_path_list.append(tlp)

""" Data parsing """

tracks = []

for tlp in track_log_path_list:
    
    track = track_log_parsing(tlp)
    tracks.append(track)
    
    """ Classification visualization """
wo_time = np.array(tracks[0]['timestamp']) - tracks[0]['timestamp'][0]
w_time = np.array(tracks[1]['timestamp']) - tracks[1]['timestamp'][0]

wo_acc_cls = np.array( tracks[0]['class'] )
w_acc_cls = np.array( tracks[1]['class'] )
wo_acc_cls[wo_acc_cls == 2] = 0
w_acc_cls[w_acc_cls == 2] = 0

%matplotlib qt
plt.close("all")
fig = plt.figure()

ax = fig.add_subplot(111)
plt.subplots_adjust(left=0.1, right=0.9, top=0.7, bottom=0.3)

ax.axvspan(0, 42, ymin=0.5, ymax=0.8, alpha=0.2, color='green', label="ground truth")

ax.fill_between(wo_time, wo_acc_cls + 0.1, step="pre", alpha=0.5)
ax.plot(wo_time, wo_acc_cls + 0.1, drawstyle="steps", label = 'w/o accumulation', marker = 'o', markersize = '5', linewidth = 1)

ax.fill_between(w_time, w_acc_cls, step="pre", alpha=0.1)
ax.plot(w_time, w_acc_cls, drawstyle="steps", color='red',label = 'w/ accumulation', marker = 's', markersize = '5', linewidth = 1)

ax.set_yticks([0, 0.4 , 1.0, 1.5])
ax.grid()
ax.set_title("Classification results - scenario1 (real data)", weight='bold')
ax.set_xlabel('time[sec]', weight='bold')
ax.set_xlim(0,9)
ax.set_ylabel('Class', fontsize=12, weight='bold')
ax.set_yticklabels( ['', 'unknown', 'car'], rotation=90)

plt.legend()
plt.show()

wo_acc_cls = wo_acc_cls[wo_time < 9.]
w_acc_cls = w_acc_cls[w_time < 9.]

print "wo classification accuracy: {}".format( float ( np.sum(wo_acc_cls == 1) ) / float ( wo_acc_cls.shape[0]) * 100.0) 
print "w classification accuracy: {}".format( float ( np.sum(w_acc_cls == 1) ) / float ( w_acc_cls.shape[0]) * 100.0) 

print "wo classification accuracy: {} / {}".format( float ( np.sum(wo_acc_cls == 1) ) , float ( wo_acc_cls.shape[0]) )
print "w classification accuracy: {} / {}".format( float ( np.sum(w_acc_cls == 1) ) , float ( w_acc_cls.shape[0]) )


wo classification accuracy: 50.0
w classification accuracy: 75.9493670886
wo classification accuracy: 39.0 / 78.0
w classification accuracy: 60.0 / 79.0


81
81


In [9]:
""" Accumulation vs accuracy """

""" Path """

track_log_path_list = []

tlp = os.path.abspath('./log_data8' )
track_log_path_list.append(tlp)


""" Data parsing """

tracks = []

for tlp in track_log_path_list:
        
    ## read all txt files
    for root, dirs, files in os.walk(tlp):
        rootpath = os.path.join(os.path.abspath(tlp), root)

        for file in files:
            if file.split('.')[-1] == 'txt':

                filepath = os.path.join(rootpath, file)
                track = track_log_parsing(filepath)
                
                tracks.append(track)
                
""" Data analysis """
t_num = [ 0. for _ in range(9)]
f_num = [ 0. for _ in range(9)]


## wo registration
for t in tracks:
    
    for i, c in enumerate( t['class'] ):
        
        if t['pointnum'][i]  < 30: continue
        
        if int( c ) == 1:
            if len(t_num) > int(t['accumulatedframe'][i]): 
                t_num[  int( t['accumulatedframe'][i] ) -1 ] +=1.
        else:
            if len(f_num) > int(t['accumulatedframe'][i]): 
                f_num[  int( t['accumulatedframe'][i] ) -1 ] +=1.
      
results = []
for i in range( len(t_num)):
    
    if float(t_num[i] + f_num[i]) == 0 : continue
        
    results.append( float(t_num[i]) / float(t_num[i] + f_num[i]) )
    print "results {}: {} / {} ".format(i+1, t_num[i] , t_num[i] +f_num[i] )

%matplotlib qt
plt.close("all")
fig = plt.figure()

ax = fig.add_subplot(111)

index = range(1, len(results) + 1)
ax.bar(index, results, align='center')
ax.grid()
ax.set_title("'Accuracy' vs '# of accumulated frames'", weight='bold')

# ax.set_xlabel('time[sec]', weight='bold')
ax.set_xlim(0,9)
# ax.set_ylabel('Class', fontsize=12, weight='bold')
# ax.set_yticklabels( ['', 'unknown', 'car', 'truck'], rotation=90)

plt.ylabel('Accuracy', weight='bold')
plt.xlabel('# of accumulated frames', weight='bold')
# plt.xlim( 0,42)
# plt.ylim( 0, np.max(y_val) * 1.1 )

plt.legend()
plt.show()

results 1: 179.0 / 234.0 
results 2: 0.0 / 1.0 
results 3: 0.0 / 1.0 
results 4: 1.0 / 1.0 
results 5: 1.0 / 1.0 
results 6: 1.0 / 1.0 
results 7: 1.0 / 1.0 
results 8: 1.0 / 1.0 


In [30]:
""" Classification vs time """

""" Path """

track_log_path_list = []

tlp = os.path.abspath('./log_data8/log_track_1.txt')
track_log_path_list.append(tlp)

tlp = os.path.abspath('./log_data8/log_track_0.txt')
track_log_path_list.append(tlp)

""" Data parsing """

tracks = []

for tlp in track_log_path_list:
    
    track = track_log_parsing(tlp)
    tracks.append(track)
    
""" Classification visualization """

wo_x = np.array(tracks[0]['x'])
wo_y = np.array(tracks[0]['y'])

w_x = np.array(tracks[1]['x'])
w_y = np.array(tracks[1]['y'])

wo_dist = np.sqrt(wo_x**2 + wo_y**2)
w_dist = np.sqrt(w_x**2 + w_y**2)

wo_acc_cls = np.array( tracks[0]['class'] )
w_acc_cls = np.array( tracks[1]['class'] )
wo_acc_cls[wo_acc_cls == 2] = 0
w_acc_cls[w_acc_cls == 2] = 0

n = 6.

wo_dist_hist = [0. for _ in range(int(n))]
w_dist_hist = [0. for _ in range(int(n))]

start_val = 7.
end_val = 25.
step = (end_val - start_val) / (n)

for i in range(int(n)):
    
    for j in range(wo_dist.shape[0]):
        if wo_dist[j] > start_val + i * step and wo_dist[j] > start_val + (i+1.) * step:
            wo_dist_hist[i] += wo_acc_cls[j]
            
    for j in range(w_dist.shape[0]):
        if w_dist[j] > start_val + i * step and w_dist[j] > start_val + (i+1.) * step:
            w_dist_hist[i] += w_acc_cls[j]
            
print wo_dist_hist
print w_dist_hist

%matplotlib qt
plt.close("all")
fig = plt.figure()

ax = fig.add_subplot(111)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
ax.plot(wo_dist_hist)
ax.plot(w_dist_hist)

# plt.show()
# ax.axvspan(0, 42, ymin=0.5, ymax=0.8, alpha=0.2, color='green', label="ground truth")

# ax.fill_between(wo_time, wo_acc_cls + 0.1, step="pre", alpha=0.5)
# ax.plot(wo_time, wo_acc_cls + 0.1, drawstyle="steps", label = 'w/o accumulation', marker = 'o', markersize = '5', linewidth = 1)

# ax.fill_between(w_time, w_acc_cls, step="pre", alpha=0.1)
# ax.plot(w_time, w_acc_cls, drawstyle="steps", color='red',label = 'w/ accumulation', marker = 's', markersize = '5', linewidth = 1)

# ax.set_yticks([0, 0.4 , 1.0, 1.5])
# ax.grid()
# ax.set_title("Classification results - scenario1", weight='bold')
# ax.set_xlabel('time[sec]', weight='bold')
# ax.set_xlim(0,10)
# ax.set_ylabel('Class', fontsize=12, weight='bold')
# ax.set_yticklabels( ['', 'unknown', 'car'], rotation=90)

# plt.legend()
# plt.show()

# print "wo classification accuracy: {}".format( float ( np.sum(wo_acc_cls == 1) ) / float ( wo_acc_cls.shape[0]) * 100.0) 
# print "w classification accuracy: {}".format( float ( np.sum(w_acc_cls == 1) ) / float ( w_acc_cls.shape[0]) * 100.0) 

# print "wo classification accuracy: {} / {}".format( float ( np.sum(wo_acc_cls == 1) ) , float ( wo_acc_cls.shape[0]) )
# print "w classification accuracy: {} / {}".format( float ( np.sum(w_acc_cls == 1) ) , float ( w_acc_cls.shape[0]) )


[142.0, 118.0, 100.0, 77.0, 40.0, 0.0]
[139.0, 115.0, 97.0, 76.0, 48.0, 5.0]


In [14]:
(16.-7.) /4.


2.25